In [4]:
import json
# !pip install gradio
import gradio as gr


In [5]:
import os

In [6]:
# Load the summaries from the JSON file
with open('movie_summaries.json', 'r') as f:
    movie_summaries = json.load(f)

def get_movie_summary(movie_title):
    # Fetch the summary for the given movie title
    summary = movie_summaries.get(movie_title, "Sorry, no summary available for this movie.")
    return summary
# Get a list of movie titles for the dropdown
movie_titles = list(movie_summaries.keys())

In [8]:
# Create a Gradio interface
title_dropdown = gr.Dropdown(choices=movie_titles, label="Select a movie title")
summary_output = gr.Textbox(label="Movie Summary")

interface = gr.Interface(fn=get_movie_summary, 
                         inputs=title_dropdown, 
                         outputs=summary_output, 
                         title="Movie Review Summarizer", 
                         theme="base",
                         css= ".gradio-container {background: url('file=film_reel.png'), #f0f0f0; background-size: cover;}",
                         description="Select a movie title to get a summary of reviews.")

# Launch the interface
interface.launch(allowed_paths=[os.path.dirname('film_reel.png')])

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
